In [39]:
# import torch
# from transformers import pipeline
# from transformers import AutoTokenizer, AutoModelForMaskedLM


# from transformers import logging
# logging.set_verbosity_error() #change 'error' to 'warning' or remove this if you want to see the warning

# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")


# from transformers import BertTokenizer, BertModel
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained("bert-base-uncased")
# text = "Replace me by any text you'd like."
# encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)

In [90]:
from transformers import pipeline
from transformers import logging
logging.set_verbosity_error() #change'error' to 'warning' or remove this if you want to see the warning
import numpy as np
unmasker = pipeline('fill-mask', model='bert-base-uncased')

from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")


In [160]:
def levenshtein_distance(s, t):
    m, n = len(s), len(t)
    d = [[0] * (n+1) for _ in range(m+1)]
    
    for i in range(m+1):
        d[i][0] = i
    
    for j in range(n+1):
        d[0][j] = j
        
    for j in range(1, n+1):
        for i in range(1, m+1):
            if s[i-1] == t[j-1]:
                d[i][j] = d[i-1][j-1]
            else:
                d[i][j] = 1 + min(d[i-1][j], d[i][j-1], d[i-1][j-1])
                
    return d[m][n]

3

In [244]:
input = "the b-movie by jerry sinfild the sound of buzzing bees can be heard according to all known laws of aviation that is no way or b t be able to fly its wings are too small to get its sart little body off the groun the be of course flies anywhay because bees don't care what humans think is possible barry benson barry is picking out ashort yellow black yellow black yellow black yellow black oh black-and-yellowlets shake it up a little janet benson baddy breakfast is ready bady coming hang on a second".split(" ")

for t in range(1):
    for i in range(len(input)):
        temp = input[i]
        token = tokenizer(temp)['input_ids'][1]
        input[i] = "[MASK]"
        apiint = unmasker(' '.join(input))
        dist = []
        for r in range(5):
            # if (np.abs((apiint[r]['token'] - token)) < 2):
            dist.append(levenshtein_distance(temp, apiint[r]['token_str']))
            # print(dist)
        lindex = 0
        l = dist[0]
        for r in range(5):
            if dist[r] < l:
                lindex = r
                l = dist[r]
        if l <= 2:
            input[i] = apiint[lindex]['token_str']
        else:
            input[i] = temp
            # if (levenshtein_distance(temp, apiint[r]['token_str']) <=2):
            #     input[i] = apiint[r]['token_str']
            #     break
            #     # print(apiint[r]['token_str'])
            # else:
            #     input[i] = temp

' '.join(input)

"the b-movie by jerry sinfild the sound of buzzing bees can be heard according to all known laws of aviation that : o a o b to be able to fly its wings are too small to get its fat little body off the ground to be of course flies anywhay because bees don't care what humans think is possible barry benson barry is picking up ashort yellow black yellow black yellow black yellow black - black-and-yellowlets shake it up a little janet benson baddy breakfast is ready bady coming hang on a second"

In [248]:
unmasker("of course flies [MASK] because")

[{'score': 0.4876376986503601,
  'token': 1010,
  'token_str': ',',
  'sequence': 'of course flies, because'},
 {'score': 0.04158661514520645,
  'token': 2185,
  'token_str': 'away',
  'sequence': 'of course flies away because'},
 {'score': 0.02643710933625698,
  'token': 2011,
  'token_str': 'by',
  'sequence': 'of course flies by because'},
 {'score': 0.016663962975144386,
  'token': 3435,
  'token_str': 'fast',
  'sequence': 'of course flies fast because'},
 {'score': 0.016079826280474663,
  'token': 1012,
  'token_str': '.',
  'sequence': 'of course flies. because'}]

In [274]:
def collation(input):
    pun_marks = [",", ".", "?", "!", ";", ":", "-", "—", "(", ")", "[", "]", "{", "}", "'", "\"", "`"]
    output = ""
    Capital = True
    for i in range(len(input)):
        if input[i] in pun_marks:
            output += input[i]
            if input[i] in [".", "("]:
                Capital = True
        elif Capital:
            output += " " 
            output += input[i].capitalize()
            Capital = False
        else:
            output += " " 
            output += input[i]
    return output

In [264]:
input = "the b-movie by jerry sinfild the sound of buzzing bees can be heard according to all known laws of aviation that : o a o b to be able to fly its wings are too small to get its fat little body off the ground to be of course flies anywhay because bees don't care what humans think is possible barry benson barry is picking up ashort yellow black yellow black yellow black yellow black - black-and-yellowlets shake it up a little janet benson baddy breakfast is ready bady coming hang on a second".split(" ")

for t in range(1):
    inndex = 1
    for i in range(len(input)):
        input.insert(inndex, "[MASK]")
        # print(' '.join(input))
        apiint = unmasker(' '.join(input))
        if (apiint[0]['token'] < 1500):
            input[inndex] = apiint[0]["token_str"]
            inndex += 2
        else:
            del input[inndex]
            inndex += 1


# ' '.join(input)

''

In [275]:
collation(input)

" The b-movie by: jerry sinfild. The sound of buzzing bees can be heard according to all known laws, of aviation, that: o o b a o o o b to be able to fly its wings are too small to get its fat little body off the ground to be of course flies anywhay, because bees don't care what humans think is possible. Barry benson. Barry is picking up ashort yellow- black- yellow- black- yellow- black- yellow- black--- black-and-yellowlets shake- it- up a little. Janet benson. Baddy breakfast is ready. Bady coming. Hang on a second."

In [225]:
# IS: 2003
# AS: 2004
unmasker("[MASK] Hello, my name is Hrsh")[0]

{'score': 0.9144755005836487,
 'token': 1000,
 'token_str': '"',
 'sequence': '" hello, my name is hrsh'}

In [278]:
# tokenizer('anywhay')
tokenizer('anyway')

{'input_ids': [101, 4312, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}